# Script to compute the kinematics (for the first time)

specifically: we compute knee, hip, elbow and arm angle with rtmo, and the ankle angle with body26 (since rtmo has no foot keypoint)

eventually, we store the data to be able to load it directly without recomputing joint angles

In [1]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import pandas as pd
import pingouin as pg
import seaborn as sns
from biopsykit.questionnaires.utils import compute_scores, wide_to_long
from biopsykit.utils.dataframe_handling import convert_nan
from fau_colors import cmaps, register_fausans_font
from scipy.interpolate import interp1d

from stressgait_analysis.dataset import StressGaitDataset

from src.stressgait_analysis.gait_helper_functions import compute_leg_kinematics, compute_arm_kinematics

%matplotlib widget
%load_ext autoreload
%autoreload 2
from stressgait_analysis.gait_helper_functions import compute_HS, resample
import numpy as np

In [2]:
deploy_type = "local"

config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
gait_data_path = Path(config_dict[deploy_type]["gait_data_path"])

In [3]:
dataset = StressGaitDataset(base_path, coarse_condition=True, gait_data_path=gait_data_path, specify_bouts=True, specify_speed=True)

In [14]:

data = []
for trial in dataset:

    keypoints = trial.load_keypoint_trajectories()
    hs_events = compute_HS(keypoints)
    n_cycle = 0
    #iterate over individual cycles:
    for i in range(len(hs_events)-1):

        cycle_start = hs_events[i]
        if cycle_start < 30:
            continue
        #print(cycle_start)
        cycle_end = hs_events[i+1]
        leg_kinematics = compute_leg_kinematics(keypoints[cycle_start:cycle_end])
        arm_kinematics = compute_arm_kinematics(keypoints[cycle_start:cycle_end])

        #resample to 1900
        leg_kinematics = resample(leg_kinematics, n=101)
        arm_kinematics = resample(arm_kinematics, n=101)

        data.extend([
             (*trial.index.values[0], n_cycle, j, *leg_row, *arm_row)
                for j, (leg_row, arm_row) in enumerate(zip(leg_kinematics.values, arm_kinematics.values))
                ])
        n_cycle += 1

In [15]:
kinematics_df = pd.DataFrame(data, columns=[*dataset.index.columns, "cycle_idx", "percentage_of_stride", *leg_kinematics.columns, *arm_kinematics.columns],)
kinematics_df.set_index([*dataset.index.columns, "cycle_idx", "percentage_of_stride"], inplace=True)

In [16]:
kinematics_df

hip_flexion  \
participant condition bout speed cycle_idx percentage_of_stride                
VP_01       omc       1    slow  0         0                       15.617907   
                                           1                       14.898942   
                                           2                       14.222986   
                                           3                       13.644998   
                                           4                       13.199635   
...                                                                      ...   
VP_47       omc       2    fast  200       96                      19.179123   
                                           97                      19.724742   
                                           98                      20.170698   
                                           99                      20.258313   
                                           100                     20.283606   

                                                                 knee_flexion  \
participant condition bout speed cycle_idx percentage_of_stride                 
VP_01       omc       1    slow  0         0                         5.708224   
                                           1                         5.079829   
                                           2                         4.740407   
                                           3                         5.059201   
                                           4                         5.675398   
...                                                                       ...   
VP_47       omc       2    fast  200       96                       13.579015   
                                           97                       11.883767   
                                           98                       10.334821   
                                           99                        9.310462   
                                           100                       8.377337   

                                                                 ankle_plantarflexion  \
participant condition bout speed cycle_idx percentage_of_stride                         
VP_01       omc       1    slow  0         0                                      NaN   
                                           1                                      NaN   
                                           2                                      NaN   
                                           3                                      NaN   
                                           4                                      NaN   
...                                                                               ...   
VP_47       omc       2    fast  200       96                                     NaN   
                                           97                                     NaN   
                                           98                                     NaN   
                                           99                                     NaN   
                                           100                                    NaN   

                                                                 elbow_flexion_angles  \
participant condition bout speed cycle_idx percentage_of_stride                         
VP_01       omc       1    slow  0         0                                 4.229529   
                                           1                                 3.657486   
                                           2                                 3.151685   
                                           3                                 2.796768   
                                           4                                 2.593602   
...                                                                               ...   
VP_47       omc       2    fast  200       96                               11.669596   
           

In [18]:
data = []
for trial in dataset:
    keypoints_rtmo = trial.load_keypoint_trajectories()
    hs_events = compute_HS(keypoints_rtmo)
    keypoints = trial.load_keypoint_trajectories(model="body26")

    n_cycle = 0
    #iterate over individual cycles:
    for i in range(len(hs_events)-1):

        cycle_start = hs_events[i]
        if cycle_start < 30:
            continue
        #print(cycle_start)
        cycle_end = hs_events[i+1]
        leg_kinematics = compute_leg_kinematics(keypoints[cycle_start:cycle_end])

        #resample to 1900
        leg_kinematics = resample(leg_kinematics, n=101)

        data.extend([
             (*trial.index.values[0], n_cycle, j, *leg_row,)
                for j, leg_row in enumerate(leg_kinematics.values)
                ])
        n_cycle += 1

In [19]:
kinematics_df_ankle = pd.DataFrame(data, columns=[*dataset.index.columns,"cycle_idx", "percentage_of_stride", *leg_kinematics.columns],)
kinematics_df_ankle

,participant,condition,bout,speed,cycle_idx,percentage_of_stride,hip_flexion,knee_flexion,ankle_plantarflexion
0,VP_01,omc,1,slow,0,0,17.169145,6.769513,-2.004872
1,VP_01,omc,1,slow,0,1,16.674042,6.312671,-1.608928
2,VP_01,omc,1,slow,0,2,16.132795,6.080012,-1.238677
3,VP_01,omc,1,slow,0,3,15.486446,6.357995,-0.926947
4,VP_01,omc,1,slow,0,4,14.790500,6.787664,-0.578666
...,...,...,...,...,...,...,...,...,...
1549537,VP_47,omc,2,fast,200,96,20.404142,15.620470,-1.871226
1549538,VP_47,omc,2,fast,200,97,20.480588,13.614560,-2.327794
1549539,VP_47,omc,2,fast,200,98,20.573811,11.832138,-2.870392
1549540,VP_47,omc,2,fast,200,99,20.754395,10.774458,-3.360518


In [20]:
kinematics_df.loc[:,"ankle_plantarflexion"] = kinematics_df_ankle.loc[:,"ankle_plantarflexion"].values

## store the data

In [21]:
filename = dataset.base_path.joinpath("kinematics/kinematics.pkl")
kinematics_df.to_pickle(filename)


# Great! Kinematics can now be loaded directly from the dataset!


In [22]:
metadata_df = dataset.get_subset(participant="VP_01").index
metadata_df

,participant,condition,bout,speed
0,VP_01,omc,1,slow
1,VP_01,omc,2,fast


store the body 26 joint angles

In [24]:
data = []
for trial in dataset:

    keypoints = trial.load_keypoint_trajectories(model="body26")
    hs_events = compute_HS(keypoints)
    n_cycle = 0
    #iterate over individual cycles:
    for i in range(len(hs_events)-1):

        cycle_start = hs_events[i]
        if cycle_start < 30:
            continue
        #print(cycle_start)
        cycle_end = hs_events[i+1]
        leg_kinematics = compute_leg_kinematics(keypoints[cycle_start:cycle_end])
        arm_kinematics = compute_arm_kinematics(keypoints[cycle_start:cycle_end])

        #resample to 1900
        leg_kinematics = resample(leg_kinematics, n=101)
        arm_kinematics = resample(arm_kinematics, n=101)

        data.extend([
             (*trial.index.values[0], n_cycle, j, *leg_row, *arm_row)
                for j, (leg_row, arm_row) in enumerate(zip(leg_kinematics.values, arm_kinematics.values))
                ])
        n_cycle += 1

In [25]:
kinematics_df = pd.DataFrame(data, columns=[*dataset.index.columns, "cycle_idx", "percentage_of_stride", *leg_kinematics.columns, *arm_kinematics.columns],)
kinematics_df.set_index([*dataset.index.columns, "cycle_idx", "percentage_of_stride"], inplace=True)

filename = dataset.base_path.joinpath("kinematics/kinematics_body26.pkl")
kinematics_df.to_pickle(filename)